<a href="https://colab.research.google.com/github/byblis/byblis/blob/main/panel_data_ASTERIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Possible Features for Model:
* Time of Day (Morning, Afternoon, Evening, Night)
* Day of the Week
* Weather Conditions (Temperature, Wind Speed, Humidity, etc.)
* Runway Changes
* Airport-Specific Effects
* Flight Path Information (Latitude, Longitude, Altitude)
* Distance between Origin and Destination Airports

Ideas for Model Improvement:
* Consider Seasonal Trends
* Explore Hierarchical Modeling
* Investigate Additional External Data Sources (e.g., Aircraft Model Data)

Remember to carefully preprocess the data, handle missing values, and apply appropriate transformations before feeding it into your model. Additionally, document your decisions and experiments for reference.

# Importando bibliotecas

In [ ]:
import requests
import pandas as pd
from google.colab import drive
import gdown

import json

import concurrent.futures

from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from math import radians, sin, cos, sqrt, atan2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Config da API

In [ ]:
base_url = 'http://montreal.icea.decea.mil.br:5002/api/v1'
token = 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173'

Lembrar de cuidar a data inicial e final de toda query a partir de agora, que ficará definida como essa mesma para todas as tabelas, excesso a CAT62

In [ ]:
data_inicial = '2022-06-01'
data_final = '2022-06-02'
# Notem que estou pegando os dados de um dia apenas, pois são muitos e não queremos fazer queries grandes demais ainda

# BIMTRA

Aqui temos, por exemplo, hora estimada de decolagem, aeroporto de origem, destino e tempo estimado de voo

Essa parte do código chama os dados do BIMTRA. Aqui temos, por exemplo, hora estimada de decolagem, aeroporto de origem, destino e tempo estimado de voo, que vamos usar no modelo agora. Especialmente, usaremos os dados de origem e destino.

In [ ]:
bimtra_url = f'{base_url}/bimtra?token={token}&idate={data_inicial}&fdate={data_final}'
response_bimtra = requests.get(bimtra_url)

if response_bimtra.status_code == 200:
    data_bimtra = response_bimtra.json()
else:
    print('Erro na solicitação:', response_bimtra.status_code)

df_bimtra = pd.DataFrame(data_bimtra)

In [ ]:
# dt_arr = df_bimtra["dt_arr"]
# dt_dep = df_bimtra["dt_dep"]

In [ ]:
df_bimtra

,flightid,origem,destino,dt_dep,dt_arr
0,fcb2bf90345705318213ae1307c0f901,SBKP,SBRJ,1654044297000,1654046760000
1,c7c5c10716335b048f86d8c52fcba3f2,SBGR,SBRJ,1654045021000,1654047173000
2,3162de7203a972f071d5a48e8f0f4828,SBBR,SBRJ,1654043212000,1654047825000
3,8fcd243e9b9eebbe62ab145ce04b6ab5,SBKP,SBCF,1654045120000,1654047701000
4,504a62621cd231d6ab67e674ce538cd3,SBCF,SBFL,1654045238000,1654050785000
...,...,...,...,...,...
1545,7009abb39cd4e71aeabb77e7e457ca4f,SBKP,SBCT,1654194585000,1654197281000
1546,1ad257b79cfa517588beee0fc0502389,SBKP,SBCT,1654195351000,1654197839000
1547,12823dc27622be5b1213f78ab6c64536,SBPA,SBCT,1654194589000,1654198118000
1548,8ec3a4b4bc31f50f6d77ba9981befa60,SBSP,SBFL,1654179151000,1654182391000


The format 1654046760000 appears to be a Unix timestamp in milliseconds. This timestamp represents the number of milliseconds since January 1, 1970 (UTC), also known as the "Unix Epoch."

I will use Python's datetime module to convert it to seconds, and then get the difference to obtain the flight duration, in seconds, stored in the variable dur_seconds

In [ ]:
# # Convert the UNIX timestamps to datetime objects
# dt_dep = pd.to_datetime(df_bimtra['dt_dep'], unit='ms')
# dt_arr = pd.to_datetime(df_bimtra['dt_arr'], unit='ms')

# # Calculate the flight duration in seconds
# df_bimtra['dur_seconds'] = (dt_arr - dt_dep).dt.total_seconds()
# dur_seconds = df_bimtra['dur_seconds']
# df_bimtra

# # Now, 'flight_duration_seconds' column will contain the duration of each flight in seconds.

NameError: ignored

In [ ]:
# # Define an empty DataFrame
# # flight_df = pd.DataFrame(columns=['FlightID', 'DepartureTime', 'ArrivalTime'])
# flight_df = pd.DataFrame()

# def make_api_request(start_date, end_date):
#     url = "http://montreal.icea.decea.mil.br:5002/api/v1i"
#     token = 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173'
#     flight_data = requests.get(f'{url}/bimtra?token={token}&idate={start_date}&fdate={end_date}')
#     return flight_data

# def process_date_range(date_range):
#     start_date, end_date = date_range
#     flight_data = make_api_request(start_date, end_date)

#     global flight_df

#     for response_data in flight_data:
#         flight_id = flight_data['flightid']
#         departure_time = flight_data['dt_dep']
#         arrival_time = flight_data['dt_arr']

#         # Print the extracted data to check if it's correct
#         print(f'FlightID: {flight_id}, DepartureTime: {departure_time}, ArrivalTime: {arrival_time}')

#         # Create a new row for the DataFrame
#         new_row = pd.Series({'flight_id': flight_id, 'departure': departure_time, 'arrival': arrival_time})

#         # Append the new row to the DataFrame
#         flight_df = flight_df.append(new_row, ignore_index=True)

#     return flight_df

# date_ranges = [
#     ("2022-06-01", "2022-06-02"),
#     ("2022-06-03", "2022-06-04"),
#     # Add more date ranges as needed
# ]

# num_concurrent_requests = 5

# with concurrent.futures.ThreadPoolExecutor(max_workers=num_concurrent_requests) as executor:
#     executor.map(process_date_range, date_ranges)

# # Now, flight_df contains the data for all flight IDs in the range


In [ ]:
# base_url = 'http://montreal.icea.decea.mil.br:5002/api/v1'
# token = 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173'

# start_date = '2022-06-01'
# end_date = '2022-06-02'

# def make_api_request(start_date, end_date):
#     bimtra_url = f'{base_url}/bimtra?token={token}&idate={data_inicial}&fdate={data_final}'
#     response_bimtra = requests.get(bimtra_url)

#     if response_bimtra.status_code == 200:
#         data_bimtra = response_bimtra.json()
#     else:
#         print('Erro na solicitação:', response_bimtra.status_code)

#     df_bimtra = pd.DataFrame(data_bimtra)
#     return df_bimtra

# response = make_api_request(start_date, end_date)
# response.head(5)


In [ ]:
BASE_URL = 'http://montreal.icea.decea.mil.br:5002/api/v1'
token = 'a779d04f85c4bf6cfa586d30aaec57c44e9b7173'

# start_date = '2022-06-01'
# end_date = '2022-06-02'


In [ ]:
def make_api_request(start_date, end_date):
    bimtra_url = f'{BASE_URL}/bimtra?token={token}&idate={start_date}&fdate={end_date}'
    response_bimtra = requests.get(bimtra_url)
    data_bimtra = {}

    if response_bimtra.status_code == 200:
        data_bimtra = response_bimtra.json()
    else:
        print('Erro na solicitação:', response_bimtra.status_code)
    data_bimtra = json.loads(data_bimtra)
    return data_bimtra

# response_data = make_api_request(start_date, end_date)
# print(response_data)

In [ ]:
# for flight_data in response_data:
#     flight_df = pd.DataFrame()
#     flight_id = flight_data.get('flightid')
#     departure_time = flight_data.get('dt_dep')
#     arrival_time = flight_data.get('dt_arr')

#     if flight_id is not None and departure_time is not None and arrival_time is not None:
#         # Create a new row for the DataFrame
#         new_row = pd.Series({'flightid': flight_id, 'dt_dep': departure_time, 'dt_arr': arrival_time})
#         # Append the new row to the DataFrame
#         flight_df = flight_df.append(new_row, ignore_index=True)

In [ ]:
# flight_df.head(5)

In [ ]:
def process_date_range(date_range):
    start_date, end_date = date_range
    flight_data = make_api_request(start_date, end_date)

    global FLIGHT_DF

    FLIGHT_DF = {}

    f_id = flight_data['flightid']
    d_time = flight_data['dt_dep']
    a_time = flight_data['dt_arr']
    orig = flight_data['origem']
    dest = flight_data['destino']

    if f_id is not None and d_time is not None and a_time is not None:
        # Create a new row for the DataFrame
        new_row = pd.Series({'flightid': f_id, 'dt_dep': d_time, 'dt_arr': a_time, 'origem': orig, 'destino': dest})

        # Append the new row to the DataFrame
        FLIGHT_DF = FLIGHT_DF.append(new_row, ignore_index=True)

    return FLIGHT_DF

date_ranges = [
    ("2022-06-01", "2022-06-02"),
    ("2022-06-03", "2022-06-04"),
    # Add more date ranges as needed
]

num_concurrent_requests = 5

with concurrent.futures.ThreadPoolExecutor(max_workers=num_concurrent_requests) as executor:
    dfs = list(executor.map(lambda dates: make_api_request(*dates), date_ranges))
    dfs = pd.DataFrame(dfs)

In [ ]:
def process_date_range(date_range):
    start_date, end_date = date_range
    flight_data = make_api_request(start_date, end_date)

    global FLIGHT_DF

    FLIGHT_DF = {}

    f_id = flight_data['flightid']
    d_time = flight_data['dt_dep']
    a_time = flight_data['dt_arr']
    orig = flight_data['origem']
    dest = flight_data['destino']

    if f_id is not None and d_time is not None and a_time is not None:
        # Create a new row for the DataFrame
        new_row = pd.Series({'flightid': f_id, 'dt_dep': d_time, 'dt_arr': a_time, 'origem': orig, 'destino': dest})

        # Append the new row to the DataFrame
        FLIGHT_DF = FLIGHT_DF.append(new_row, ignore_index=True)

    return FLIGHT_DF

date_ranges = [
    ("2022-06-01", "2022-06-02"),
    ("2022-06-03", "2022-06-04"),
    # Add more date ranges as needed
]

num_concurrent_requests = 5

with concurrent.futures.ThreadPoolExecutor(max_workers=num_concurrent_requests) as executor:
    dfs = list(executor.map(lambda dates: make_api_request(*dates), date_ranges))
    dfs = pd.DataFrame(dfs)

dfs.head(5)

In [ ]:
df = process_date_range(date_ranges[0])
df.head(3)

TypeError: ignored

In [ ]:
dfs.describe()

,0,1,2,3,4,5,6,7,8,9,...,1540,1541,1542,1543,1544,1545,1546,1547,1548,1549
count,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
unique,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
top,{'flightid': 'fcb2bf90345705318213ae1307c0f901...,{'flightid': 'c7c5c10716335b048f86d8c52fcba3f2...,{'flightid': '3162de7203a972f071d5a48e8f0f4828...,{'flightid': '8fcd243e9b9eebbe62ab145ce04b6ab5...,{'flightid': '504a62621cd231d6ab67e674ce538cd3...,{'flightid': 'a94ac7165dec388cf950b86e95d85e12...,{'flightid': '052050016538ab736a254313ef009a6b...,{'flightid': '9ee544f8f82e5036fbdf6984e918308b...,{'flightid': 'e93e3b8510854a55f195788019b5763b...,{'flightid': '2386fcd950814c8d343d93c4b51585d1...,...,{'flightid': 'cdd298c48813b226a0ceaa00993838fb...,{'flightid': '6d515706db336d4ad9f95deb448dffcc...,{'flightid': '050a0e06040385945e858f67e752305f...,{'flightid': '83f6508b270c0781e871845beb328adc...,{'flightid': '4d81af26ccc6934fefd1363b7c662bd1...,{'flightid': '7009abb39cd4e71aeabb77e7e457ca4f...,{'flightid': '1ad257b79cfa517588beee0fc0502389...,{'flightid': '12823dc27622be5b1213f78ab6c64536...,{'flightid': '8ec3a4b4bc31f50f6d77ba9981befa60...,{'flightid': '03ca0a757c3b376224c86862ccca8008...
freq,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
dfs.T

,0,1
0,{'flightid': 'fcb2bf90345705318213ae1307c0f901...,{'flightid': '310b5374273eb182ca50015a2563ac65...
1,{'flightid': 'c7c5c10716335b048f86d8c52fcba3f2...,{'flightid': 'fb268a95b96eb7cbe0e3b12bd4e9da28...
2,{'flightid': '3162de7203a972f071d5a48e8f0f4828...,{'flightid': 'cf718db1aaa63b12e433180a3059d583...
3,{'flightid': '8fcd243e9b9eebbe62ab145ce04b6ab5...,{'flightid': '9f83aca5c88a474f5962c9b2d87ef8a1...
4,{'flightid': '504a62621cd231d6ab67e674ce538cd3...,{'flightid': 'c85eb86292b51c3e676ea5eda516e65f...
...,...,...
1545,{'flightid': '7009abb39cd4e71aeabb77e7e457ca4f...,None
1546,{'flightid': '1ad257b79cfa517588beee0fc0502389...,None
1547,{'flightid': '12823dc27622be5b1213f78ab6c64536...,None
1548,{'flightid': '8ec3a4b4bc31f50f6d77ba9981befa60...,None


In [ ]:
# Assuming your DataFrame is named 'df' and the column with JSON objects is named 'flight_info'
# You may need to adjust these names to match your actual DataFrame

# Create a new DataFrame to store the expanded flight information
expanded_df = pd.DataFrame(columns=['flightid', 'origem', 'destino', 'dt_dep', 'dt_arr'])

# Iterate over each row in the original DataFrame
for index, row in dfs.iterrows():
    # Extract the JSON object from the 'flight_info' column
    flight_json = row['flight_info']

    # Create a new DataFrame with the flight information from the JSON
    flight_df = pd.DataFrame([flight_json])

    # Append the flight DataFrame to the expanded DataFrame
    expanded_df = pd.concat([expanded_df, flight_df], ignore_index=True)

# Now 'expanded_df' contains a row for each flight


KeyError: ignored

In [ ]:
expanded_df

NameError: ignored

In [ ]:
# print(flightdf.iloc[0])

0       {'flightid': 'fcb2bf90345705318213ae1307c0f901', 'origem': 'SBKP', 'destino': 'SBRJ', 'dt_dep': 1654044297000, 'dt_arr': 1654046760000}
1       {'flightid': 'c7c5c10716335b048f86d8c52fcba3f2', 'origem': 'SBGR', 'destino': 'SBRJ', 'dt_dep': 1654045021000, 'dt_arr': 1654047173000}
2       {'flightid': '3162de7203a972f071d5a48e8f0f4828', 'origem': 'SBBR', 'destino': 'SBRJ', 'dt_dep': 1654043212000, 'dt_arr': 1654047825000}
3       {'flightid': '8fcd243e9b9eebbe62ab145ce04b6ab5', 'origem': 'SBKP', 'destino': 'SBCF', 'dt_dep': 1654045120000, 'dt_arr': 1654047701000}
4       {'flightid': '504a62621cd231d6ab67e674ce538cd3', 'origem': 'SBCF', 'destino': 'SBFL', 'dt_dep': 1654045238000, 'dt_arr': 1654050785000}
                                                                         ...                                                                   
1545    {'flightid': '7009abb39cd4e71aeabb77e7e457ca4f', 'origem': 'SBKP', 'destino': 'SBCT', 'dt_dep': 1654194585000, 'dt_arr': 1654197

In [ ]:
# # Assuming 'df' is your DataFrame
# pd.set_option('display.max_colwidth', None)

# # Now, when you print 'df', the contents of each cell won't be truncated
# flightdf


,0,1,2,3,4,5,6,7,8,9,...,1540,1541,1542,1543,1544,1545,1546,1547,1548,1549
0,"{'flightid': 'fcb2bf90345705318213ae1307c0f901', 'origem': 'SBKP', 'destino': 'SBRJ', 'dt_dep': 1654044297000, 'dt_arr': 1654046760000}","{'flightid': 'c7c5c10716335b048f86d8c52fcba3f2', 'origem': 'SBGR', 'destino': 'SBRJ', 'dt_dep': 1654045021000, 'dt_arr': 1654047173000}","{'flightid': '3162de7203a972f071d5a48e8f0f4828', 'origem': 'SBBR', 'destino': 'SBRJ', 'dt_dep': 1654043212000, 'dt_arr': 1654047825000}","{'flightid': '8fcd243e9b9eebbe62ab145ce04b6ab5', 'origem': 'SBKP', 'destino': 'SBCF', 'dt_dep': 1654045120000, 'dt_arr': 1654047701000}","{'flightid': '504a62621cd231d6ab67e674ce538cd3', 'origem': 'SBCF', 'destino': 'SBFL', 'dt_dep': 1654045238000, 'dt_arr': 1654050785000}","{'flightid': 'a94ac7165dec388cf950b86e95d85e12', 'origem': 'SBSP', 'destino': 'SBCT', 'dt_dep': 1654042531000, 'dt_arr': 1654045155000}","{'flightid': '052050016538ab736a254313ef009a6b', 'origem': 'SBRJ', 'destino': 'SBCF', 'dt_dep': 1654075204000, 'dt_arr': 1654078210000}","{'flightid': '9ee544f8f82e5036fbdf6984e918308b', 'origem': 'SBSP', 'destino': 'SBPA', 'dt_dep': 1654045193000, 'dt_arr': 1654050232000}","{'flightid': 'e93e3b8510854a55f195788019b5763b', 'origem': 'SBGR', 'destino': 'SBPA', 'dt_dep': 1654046445000, 'dt_arr': 1654051465000}","{'flightid': '2386fcd950814c8d343d93c4b51585d1', 'origem': 'SBGR', 'destino': 'SBPA', 'dt_dep': 1654046039000, 'dt_arr': 1654051301000}",...,"{'flightid': 'cdd298c48813b226a0ceaa00993838fb', 'origem': 'SBKP', 'destino': 'SBSV', 'dt_dep': 1654160153000, 'dt_arr': 1654166939000}","{'flightid': '6d515706db336d4ad9f95deb448dffcc', 'origem': 'SBRJ', 'destino': 'SBCT', 'dt_dep': 1654185807000, 'dt_arr': 1654190363000}","{'flightid': '050a0e06040385945e858f67e752305f', 'origem': 'SBSP', 'destino': 'SBCT', 'dt_dep': 1654188764000, 'dt_arr': 1654191275000}","{'flightid': '83f6508b270c0781e871845beb328adc', 'origem': 'SBCF', 'destino': 'SBCT', 'dt_dep': 1654189449000, 'dt_arr': 1654194489000}","{'flightid': '4d81af26ccc6934fefd1363b7c662bd1', 'origem': 'SBSP', 'destino': 'SBCT', 'dt_dep': 1654193375000, 'dt_arr': 1654195834000}","{'flightid': '7009abb39cd4e71aeabb77e7e457ca4f', 'origem': 'SBKP', 'destino': 'SBCT', 'dt_dep': 1654194585000, 'dt_arr': 1654197281000}","{'flightid': '1ad257b79cfa517588beee0fc0502389', 'origem': 'SBKP', 'destino': 'SBCT', 'dt_dep': 1654195351000, 'dt_arr': 1654197839000}","{'flightid': '12823dc27622be5b1213f78ab6c64536', 'origem': 'SBPA', 'destino': 'SBCT', 'dt_dep': 1654194589000, 'dt_arr': 1654198118000}","{'flightid': '8ec3a4b4bc31f50f6d77ba9981befa60', 'origem': 'SBSP', 'destino': 'SBFL', 'dt_dep': 1654179151000, 'dt_arr': 1654182391000}","{'flightid': '03ca0a757c3b376224c86862ccca8008', 'origem': 'SBRJ', 'destino': 'SBSP', 'dt_dep': 1654184150000, 'dt_arr': 1654187147000}"
1,"{'flightid': '310b5374273eb182ca50015a2563ac65', 'origem': 'SBRJ', 'destino': 'SBSP', 'dt_dep': 1654300104000, 'dt_arr': 1654298716000}","{'flightid': 'fb268a95b96eb7cbe0e3b12bd4e9da28', 'origem': 'SBRJ', 'destino': 'SBSP', 'dt_dep': 1654298696000, 'dt_arr': 1654298716000}","{'flightid': 'cf718db1aaa63b12e433180a3059d583', 'origem': 'SBGR', 'destino': 'SBRJ', 'dt_dep': 1654297955000, 'dt_arr': 1654300024000}","{'flightid': '9f83aca5c88a474f5962c9b2d87ef8a1', 'origem': 'SBFL', 'destino': 'SBKP', 'dt_dep': 1654272537000, 'dt_arr': 1654275970000}","{'flightid': 'c85eb86292b51c3e676ea5eda516e65f', 'origem': 'SBCF', 'destino': 'SBKP', 'dt_dep': 1654273712000, 'dt_arr': 1654277109000}","{'flightid': '64ef5cd253e1799f5eb429f39c6de269', 'origem': 'SBRF', 'destino': 'SBSP', 'dt_dep': 1654276270000, 'dt_arr': 1654287351000}","{'flightid': '1826a53aa03f569be38da667ea196521', 'origem': 'SBRJ', 'destino': 'SBSP', 'dt_dep': 1654284385000, 'dt_arr': 1654287965000}","{'flightid': '1a71c15252cdbf49da2034e0aed1c742', 'origem': 'SBSV', 'destino': 'SBSP', 'dt_dep': 1654278375000, 'dt_arr': 1654286872000}","{'flightid': 'a89e2755b36338f6424ac12c6fb3c824', 'or

In [ ]:
# flight_data = pd.DataFrame(flight_data)

# CAT62

Dados do espaço aéreo brasileiro, a cada 4 segundos (CUIDAR TAMANHO DA QUERY)

Os dados da base de dados CAT62 são dados a cada 4 segundos, como flight id, lat, lon, altitude, velocidade, data e tempo, por radares, do espaço aéreo brasileiro. Por isso, cuidar a quantidade de dados que a query vai puxar, uma hora de dados já tem quase 7 mil linhas

In [ ]:
data_inicial_cat62 = '2022-06-02 18:30:00.000'
data_final_cat62 = '2022-06-02 19:30:00.000'
cat62_url = f'{base_url}/cat-62?token={token}&idate={data_inicial_cat62}&fdate={data_final_cat62}'
response_cat62 = requests.get(cat62_url)

if response_cat62.status_code == 200:
    data_cat62 = response_cat62.json()
else:
    print('Erro na solicitação:', response_cat62.status_code)

df_cat62 = pd.DataFrame(data_cat62)
df_cat62

,flightid,lat,lon,flightlevel,speed,dt_radar
0,31388588094ea773c5d855a95f70eefc,-0.463859,-0.844314,306.0,329.0,1654197301000
1,31388588094ea773c5d855a95f70eefc,-0.469943,-0.844931,267.0,296.0,1654197064000
2,31388588094ea773c5d855a95f70eefc,-0.478863,-0.845464,129.0,288.0,1654196702000
3,31388588094ea773c5d855a95f70eefc,-0.477360,-0.845316,155.0,302.0,1654196762000
4,31388588094ea773c5d855a95f70eefc,-0.480283,-0.845639,105.0,274.0,1654196641000
...,...,...,...,...,...,...
6570,79f30e2a82a745039411c67a1ec004da,-0.413395,-0.807500,172.0,372.0,1654198200000
6571,79f30e2a82a745039411c67a1ec004da,-0.413660,-0.809359,134.0,360.0,1654198143000
6572,79f30e2a82a745039411c67a1ec004da,-0.413970,-0.811204,102.0,318.0,1654198083000
6573,79f30e2a82a745039411c67a1ec004da,-0.413350,-0.812750,65.0,262.0,1654198023000


# Esperas

Sobre os dados de espera em vôo: (não entendi o que são)

Lembrar de cuidar a data inicial e final da query de novo:

In [ ]:
data_inicial = '2022-06-01'
data_final = '2022-06-02'

Sobre os dados de espera em vôo: (não entendi o que são)

In [ ]:
esperas_url = f'{base_url}/esperas?token={token}&idate={data_inicial}&fdate={data_final}'
response_esperas = requests.get(esperas_url)

if response_esperas.status_code == 200:
    data_esperas = response_esperas.json()
else:
    print('Erro na solicitação:', response_esperas.status_code)

df_esperas = pd.DataFrame(data_esperas)
df_esperas

,esperas,hora,aero
0,0,1654041600000,SBBR
1,0,1654045200000,SBBR
2,0,1654048800000,SBBR
3,0,1654052400000,SBBR
4,0,1654056000000,SBBR
...,...,...,...
571,0,1654196400000,SBSV
572,0,1654200000000,SBSV
573,0,1654203600000,SBSV
574,0,1654207200000,SBSV


# METAF

Os dados do METAF são de previsões meteorológicas (F de forecast). Temos dados das condições do vento, visibilidade, nuvens, etc

- METAF: Indicates that this is a METAR or routine aviation weather observation.
- SBGL: This is the ICAO code for the airport (in this case, it refers to Rio de Janeiro/Galeão – Antônio Carlos Jobim International Airport).
- 010000Z: This part of the METAF refers to the date and time of the observation. In this case, it indicates the first day of the month at 00:00 UTC (Coordinated Universal Time).
- 25002KT: Describes the wind conditions. It indicates that the wind is blowing from 250 degrees at a speed of 2 knots.
- 3000: Represents the visibility. It states that the visibility is 3000 meters.
- BR: Indicates that there is mist (or "mist" is occurring).
- OVC033: Describes the cloud cover. It means that the sky is overcast at an altitude of 3300 feet above ground level.

In summary, this METAF provides weather information for each airport, including details about wind, visibility, weather phenomena (mist), and cloud cover.

Please note that decoding METAR and METAF reports can involve a bit of complex parsing, and specialized libraries or tools can be used for this purpose in data processing pipelines.

- 010000Z: This is the date and time of the forecast. The format is day of the month (01), hour (00), and minutes (00) in UTC (Z stands for Zulu time, which is equivalent to UTC).
- 25002KT:
  - 250: Wind direction in degrees true (in this case, 250 degrees).
  - 02: Wind speed in knots (in this case, 2 knots).
  - 3000: Visibility in meters (in this case, 3000 meters).
- BR: Weather condition. "BR" indicates mist.
- OVC033:
  - OVC: Overcast. The sky is completely covered by clouds.
  - 033: The cloud base is at 3300 feet above ground level.
- 23/21: Temperature and Dew Point.
  - 23: Temperature in degrees Celsius.
  - 21: Dew Point in degrees Celsius.
- Q1016: Altimeter setting. This is the atmospheric pressure at sea level. In this case, it's 1016 hPa.

The equals sign (=) at the end doesn't have a specific meaning in METAR/METAF reports and is often used as a delimiter.

This report provides crucial information for aviation, allowing pilots and other aviation professionals to understand the current and forecasted weather conditions at an airport.

In [ ]:
metaf_url = f'{base_url}/metaf?token={token}&idate={data_inicial}&fdate={data_final}'
response_metaf = requests.get(metaf_url)

if response_metaf.status_code == 200:
    data_metaf = response_metaf.json()
else:
    print('Erro na solicitação:', response_metaf.status_code)

df_metaf = pd.DataFrame(data_metaf)
df_metaf

,hora,metaf,aero
0,1654041600000,METAF SBGL 010000Z 25002KT 3000 BR OVC033...,SBGL
1,1654045200000,METAF SBGL 010100Z 02001KT 2000 BR OVC011 ...,SBGL
2,1654048800000,METAF SBGL 010200Z 09000KT 2000 BR OVC011 ...,SBGL
3,1654052400000,METAF SBGL 010300Z 09002KT 1000 RABR OVC033...,SBGL
4,1654056000000,METAF SBGL 010400Z 07003KT 1000 RABR OVC033...,SBGL
...,...,...,...
123,1654196400000,METAF SBSP 021900Z 17007KT 2000 BR OVC032...,SBSP
124,1654200000000,METAF SBSP 022000Z 16007KT 3000 BR OVC033...,SBSP
125,1654203600000,METAF SBSP 022100Z 15008KT 3000 BR OVC033...,SBSP
126,1654207200000,METAF SBSP 022200Z 15005KT 3000 BR OVC033...,SBSP


In [ ]:
pd.set_option('display.max_colwidth', None)
df_metaf["metaf"].head(20)

0     METAF SBGL 010000Z  25002KT 3000     BR OVC033   23/21 Q1016=\n
1      METAF SBGL 010100Z 02001KT 2000     BR OVC011   22/21 Q1017=\n
2      METAF SBGL 010200Z 09000KT 2000     BR OVC011   22/21 Q1018=\n
3     METAF SBGL 010300Z 09002KT 1000    RABR OVC033   22/21 Q1017=\n
4     METAF SBGL 010400Z 07003KT 1000    RABR OVC033   22/21 Q1016=\n
5     METAF SBGL 010500Z 09003KT 1000    RABR OVC033   22/21 Q1016=\n
6      METAF SBGL 010600Z 05002KT 1000     BR OVC033   22/21 Q1014=\n
7      METAF SBGL 010700Z 04004KT 1000     BR OVC033   21/21 Q1016=\n
8      METAF SBGL 010800Z 02003KT 1000     BR OVC033   21/21 Q1016=\n
9      METAF SBGL 010900Z 03005KT 1000     BR OVC011   21/21 Q1016=\n
10     METAF SBGL 011000Z 04005KT 1000     BR OVC013   22/21 Q1017=\n
11     METAF SBGL 011100Z 07003KT 2000     BR BKN033   22/20 Q1017=\n
12    METAF SBGL 011200Z  15002KT 2000     BR BKN033   22/21 Q1017=\n
13    METAF SBGL 011300Z  10002KT 4000     BR SCT014   23/20 Q1018=\n
14     METAF SBGL 01

# METAR

Os dados do METAR são os dados meteorológicos reais.


- METAR: Indicates that this is a METAR report.
- SBBR: This is the ICAO code for the airport (in this case, it refers to Brasília International Airport).
- 010000Z: This part of the METAR refers to the date and time of the observation. In this case, it indicates the first day of the month at 00:00 UTC (Coordinated Universal Time).
- 07002KT: Describes the wind conditions. It indicates that the wind is blowing from 070 degrees at a speed of 2 knots.
- CAVOK: Stands for "Ceiling and Visibility OK". It means that there are no significant clouds below 5000 feet, and visibility is 10 kilometers or more.
- 21/08: Represents the temperature and dew point. It indicates a temperature of 21°C and a dew point of 8°C.
- Q1018: Indicates the atmospheric pressure at sea level. In this case, it's 1018 hPa (hectopascal).

In summary, this METAR report provides weather information for each airport, including details about wind, visibility, temperature, and atmospheric pressure.

Similar to METAF reports, decoding METAR reports may require specialized libraries or tools for data processing.

In [ ]:
metar_url = f'{base_url}/metar?token={token}&idate={data_inicial}&fdate={data_final}'
response_metar = requests.get(metar_url)

if response_metar.status_code == 200:
    data_metar = response_metar.json()
else:
    print('Erro na solicitação:', response_metar.status_code)

df_metar = pd.DataFrame(data_metar)

In [ ]:
df_metar

,hora,metar,aero
0,1654041600000,METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=,SBBR
1,1654045200000,METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=,SBBR
2,1654048800000,METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=,SBBR
3,1654052400000,METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=,SBBR
4,1654056000000,METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=,SBBR
...,...,...,...
644,1654196400000,METAR SBSV 021900Z 14007KT 9999 SCT023 FEW026T...,SBSV
645,1654200000000,METAR SBSV 022000Z 15008KT 9999 SCT023 FEW026T...,SBSV
646,1654203600000,METAR SBSV 022100Z 15008KT 9999 SCT023 FEW026T...,SBSV
647,1654207200000,METAR SBSV 022200Z 15008KT 9999 SCT023 BKN090 ...,SBSV


In [ ]:
pd.set_option('display.max_colwidth', None)
df_metar["metar"].head(10)

0    METAR SBBR 010000Z 07002KT CAVOK 21/08 Q1018=
1    METAR SBBR 010100Z 10002KT CAVOK 20/09 Q1019=
2    METAR SBBR 010200Z 00000KT CAVOK 16/09 Q1019=
3    METAR SBBR 010300Z 27002KT CAVOK 16/09 Q1019=
4    METAR SBBR 010400Z 00000KT CAVOK 14/09 Q1018=
5    METAR SBBR 010500Z 27004KT CAVOK 13/08 Q1018=
6    METAR SBBR 010600Z 25003KT CAVOK 14/08 Q1018=
7    METAR SBBR 010700Z 28002KT CAVOK 12/08 Q1018=
8    METAR SBBR 010800Z VRB01KT CAVOK 11/08 Q1018=
9    METAR SBBR 010900Z 26002KT CAVOK 12/09 Q1019=
Name: metar, dtype: object

# Sat (imagens)

Também podemos usar dados com imagens de satélite de diversos momentos dos vôos, mas por enquanto não vamos usar imagens.

Os dados do satelite_url são o caminho para imagens de satélite em cada horário indicado. Por enquanto não vamos usar as imagens.

In [ ]:
satelite_url = f'{base_url}/satelite?token={token}&idate={data_inicial}&fdate={data_final}'
response_satelite = requests.get(satelite_url)

if response_satelite.status_code == 200:
    data_satelite = response_satelite.json()
else:
    print('Erro na solicitação:', response_satelite.status_code)

df_satelite = pd.DataFrame(data_satelite)

In [ ]:
df_satelite.head(5)

,data,path,tamanho
0,2022-06-01 01:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1879673
1,2022-06-01 02:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1877693
2,2022-06-01 03:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1887299
3,2022-06-01 04:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1890950
4,2022-06-01 05:00:00,http://satelite.cptec.inpe.br/repositoriogoes/...,1892776


In [ ]:
pd.set_option('display.max_colwidth', None)

df_satelite["path"].head(5)

0    http://satelite.cptec.inpe.br/repositoriogoes/goes16/goes16_web/ams_ret_ch11_baixa/2022/06/S11635384_202206010100.jpg
1    http://satelite.cptec.inpe.br/repositoriogoes/goes16/goes16_web/ams_ret_ch11_baixa/2022/06/S11635384_202206010200.jpg
2    http://satelite.cptec.inpe.br/repositoriogoes/goes16/goes16_web/ams_ret_ch11_baixa/2022/06/S11635384_202206010300.jpg
3    http://satelite.cptec.inpe.br/repositoriogoes/goes16/goes16_web/ams_ret_ch11_baixa/2022/06/S11635384_202206010400.jpg
4    http://satelite.cptec.inpe.br/repositoriogoes/goes16/goes16_web/ams_ret_ch11_baixa/2022/06/S11635384_202206010500.jpg
Name: path, dtype: object

# Troca de Cabeceira

Aqui temos os dados com a previsão de troca e das trocas reais de cabeceira do avião.


Todas as pistas dos aeroportos têm grandes números pintados próximos às cabeceiras. Eles são diferentes em cada extremidade da pista e têm a função de indicar aos pilotos a orientação magnética da bússola para os pousos e decolagens. O sentido dos pousos e decolagens varia de acordo com a direção do vento. Em aeroportos com torre de controle, é o controlador que orienta qual cabeceira deve ser utilizada. Essa orientação é feita ao informar o número da cabeceira em uso. Em aeroportos sem torre de controle, é o próprio piloto quem decide o sentido dos pousos e decolagens e informa, pelo rádio, o número da cabeceira que será utilizada.

Os dados df_tcprev são das previsões de troca, em bool, sendo sim ou não (1 ou 0), também dando a hora e o aerodromo.

Ideias:
- Since you're working with a time series, consider if there are any time lags between runway changes and their impact on flight durations.
- If certain airports have more frequent or complex runway changes, you may want to include airport-specific effects in your model.
- Check for correlations between runway changes and flight durations. This can be done visually through plots and quantitatively through correlation coefficients.

In [ ]:
tc_prev_url = f'{base_url}/tc-prev?token={token}&idate={data_inicial}&fdate={data_final}'
response_prev_url = requests.get(tc_prev_url)

if response_prev_url.status_code == 200:
    data_prev_url = response_prev_url.json()
else:
    print('Erro na solicitação:', response_prev_url.status_code)

df_tcprev = pd.DataFrame(data_prev_url)

In [ ]:
df_tcprev.tail(5)

,hora,troca,aero
571,1654196400000,1,SV
572,1654200000000,1,SV
573,1654203600000,0,SV
574,1654207200000,0,SV
575,1654210800000,0,SV


Já a DB do tc_real dá os horários reais da troca de cabeceira, e o número das antigas e novas.

In [ ]:
tc_real_url = f'{base_url}/tc-real?token={token}&idate={data_inicial}&fdate={data_final}'
response_real_url = requests.get(tc_real_url)

if response_real_url.status_code == 200:
    data_real_url = response_real_url.json()
else:
    print('Erro na solicitação:', response_real_url.status_code)

df_tcreal = pd.DataFrame(data_real_url)

In [ ]:
df_tcreal

,hora,nova_cabeceira,antiga_cabeceira,aero
0,1654092843000,32,03,FL
1,1654109470000,18,12,RF
2,1654110745000,16,13,CF
3,1654110758000,15,26,GL
4,1654110777000,15,26,GL
...,...,...,...,...
176,1654104436000,18,12,RF
177,1654106547000,17,10,SV
178,1654106664000,32,11,FL
179,1654107611000,10,17,SV


# Testes

In [ ]:
# # Assuming df_bimtra is your DataFrame
# columns = df_bimtra.columns
# print(columns)

Index(['flightid', 'origem', 'destino', 'dt_dep', 'dt_arr'], dtype='object')


In [ ]:
df_cat62.head(30)

,flightid,lat,lon,flightlevel,speed,dt_radar
0,31388588094ea773c5d855a95f70eefc,-0.463859,-0.844314,306.0,329.0,1654197301000
1,31388588094ea773c5d855a95f70eefc,-0.469943,-0.844931,267.0,296.0,1654197064000
2,31388588094ea773c5d855a95f70eefc,-0.478863,-0.845464,129.0,288.0,1654196702000
3,31388588094ea773c5d855a95f70eefc,-0.477360,-0.845316,155.0,302.0,1654196762000
4,31388588094ea773c5d855a95f70eefc,-0.480283,-0.845639,105.0,274.0,1654196641000
5,31388588094ea773c5d855a95f70eefc,-0.467189,-0.844650,291.0,302.0,1654197181000
6,31388588094ea773c5d855a95f70eefc,-0.481605,-0.845452,75.0,268.0,1654196581000
7,31388588094ea773c5d855a95f70eefc,-0.465373,-0.844468,304.0,308.0,1654197245000
8,31388588094ea773c5d855a95f70eefc,-0.454440,-0.843347,341.0,399.0,1654197606000
9,31388588094ea773c5d855a95f70eefc,-0.482888,-0.845154,45.0,241.0,1654196521000


In [ ]:
flight_ids = df_cat62["flightid"].unique()

In [ ]:
# flight_ids = ['flight1', 'flight2', '31388588094ea773c5d855a95f70eefc']
flight_ids = ['31388588094ea773c5d855a95f70eefc']

# Create an empty dictionary to store dataframes
flight_data = {}

for flight_id in flight_ids:
    # Define API parameters for each flight
    params = {
        'flight_id': flight_id,
        'start_date': '2022-06-02 18:30:00.000',
        'end_date': '2022-06-02 19:30:00.000'
    }

    # Make API call
    cat62_url = f'{base_url}/cat-62?token={token}&idate={data_inicial_cat62}&fdate={data_final_cat62}'
    response_cat62 = requests.get(cat62_url)
    if response_cat62.status_code == 200:
        data_cat62 = response_cat62.json()
    else:
        print('Erro na solicitação:', response_cat62.status_code)

# Now you have separate dataframes for each flight in the flight_data dictionary
# Access them like flight_data['flight1'], flight_data['flight2'], etc.

flight_data = pd.DataFrame(data_cat62)
flight_data


,flightid,lat,lon,flightlevel,speed,dt_radar
0,31388588094ea773c5d855a95f70eefc,-0.463859,-0.844314,306.0,329.0,1654197301000
1,31388588094ea773c5d855a95f70eefc,-0.469943,-0.844931,267.0,296.0,1654197064000
2,31388588094ea773c5d855a95f70eefc,-0.478863,-0.845464,129.0,288.0,1654196702000
3,31388588094ea773c5d855a95f70eefc,-0.477360,-0.845316,155.0,302.0,1654196762000
4,31388588094ea773c5d855a95f70eefc,-0.480283,-0.845639,105.0,274.0,1654196641000
...,...,...,...,...,...,...
6570,79f30e2a82a745039411c67a1ec004da,-0.413395,-0.807500,172.0,372.0,1654198200000
6571,79f30e2a82a745039411c67a1ec004da,-0.413660,-0.809359,134.0,360.0,1654198143000
6572,79f30e2a82a745039411c67a1ec004da,-0.413970,-0.811204,102.0,318.0,1654198083000
6573,79f30e2a82a745039411c67a1ec004da,-0.413350,-0.812750,65.0,262.0,1654198023000
